#web scraping thingy 

In [133]:
# ["type", "beds", "price", "location", "url", "aquisition", "contact", "duration", "furnished", "serviced", "imageurl"] //compare rent vs buy(month v day), should I buy/recommeded
# paginationSelector = soup.find('div', class_='pagination-area').nav.ul.findAll('li')[3].a['href']
# paginationSelector


In [5]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from bs4.element import Comment
import re
import csv
import json
import pandas as pd

In [2]:

def getPageData(url):
    property =  0

    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')   
    listing_property = soup.find_all("div", class_='listings-property')
    data = []

    for i in listing_property:
        if "sponsored-listing" in i["class"]:
            listing_property.remove(i)
        else:
            property = listing_property.index(i)
            amenities = listing_property[property].find('div', class_='fur-areea').findAll("span", recursive=False)
            beds = amenities[0].text[0:1]
            bathrooms = amenities[1].text[0:1]
            toilets = amenities[2].text[0:1]
            if beds == "0" or beds == 0 or beds == "":
                break
            if bathrooms == "0" or bathrooms == 0 or bathrooms == "":
                break
            if toilets == "0" or toilets == 0 or toilets == "":
                break
            agentContact = re.sub("[\n ]", "", listing_property[property].find('div', class_='phone-icon').text)
            listingTitile = listing_property[property].find('h3', class_='listings-property-title2').text
            listingURL = "https://www.propertypro.ng" + listing_property[property].find("div", "result-img").a['href']
            price = ''.join(re.findall(r'\d+', listing_property[property].find('h3', class_='listings-price').text))
            location = listing_property[property].find('div', class_='result-img').a.img['alt'].split(' ')
            street = " ".join(listing_property[property].find('div', class_='single-room-text').findAll("h4")[1].text.split(' ')[ : -2])
            if street == "":
                street = "unstated"
            city = location[-2]
            state = location[-1]
            try:
                status = str(re.sub("[\n]", "", listing_property[4].find('div', class_='furnished-btn').text))
                if status == "":
                    status = "old"
            except: 
                status = "old"
            try:
                dateUpdated = " ".join(listing_property[property].find('div', class_='single-room-text').find("h5").text.split()[-3 :])
            except:
                dateUpdated = "not stated"
            try:
                imagesProvided =  re.sub("[\n ]", "", listing_property[property].find('div', class_='room-icons').text)
            except:
                imagesProvided = 3 
            data.append([listingTitile, price, bathrooms, beds, toilets, street, city, state, status, agentContact, imagesProvided, listingURL, dateUpdated])
    return data


In [4]:
numIterations = 300
startingUrl = "https://www.propertypro.ng/property-for-rent/house/?search=&auto=&bedroom=&min_price=&max_price="
data = []

for pageNum  in range(numIterations):
    if pageNum == 0:
        url = startingUrl
    else:
        url =  startingUrl + "&page=" + str(pageNum)
    try: 
        searchResult = getPageData(url)
        data = data + searchResult
        print(pageNum)
    except Exception as e:
        print(pageNum, e)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125 'NoneType' object has no attribute 'text'
126 'NoneType' object has no attribute 'text'
127 'NoneType' object has no attribute 'text'
128
129 'NoneType' object has no attribute 'text'
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152 'NoneType' object has no attribute 'text'
153
154
155
156
157 'NoneType' object has no attribute 'text'
158 'NoneType' object has no attribute 'text'
159 'NoneType' object has no attribute 'text'
160 'NoneType' object has no attribute 'text'
161 'NoneType' object has no attribute 'text'
162
163
164
165
166
167
168
169
170
171
17

In [4]:
# WRITE DATA TO CSV FILE
header =["listingTitile", "price", "bathrooms", "beds", "toilets", "street", "city", "state", "status", "agentContact", "imagesProvided", "listingURL", "dateUpdated"]
# header = [listingTitile"," price"," bathrooms"," beds", "toilets", "street", "city", "state", "status", "agentContact", "imagesProvided", "listingURL", "dateUpdated"]
with open('HouseRent6.csv', 'w', encoding='UTF8') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(header)
    for i in data:
        writer.writerow(i)

In [6]:
propertyForSale = pd.read_csv("ScrapedSalePropertyDataSet.csv")
propertyForRent = pd.read_csv("ScrapedRentalDataSet.csv")

In [14]:
req = Request("https://www.propertypro.ng/property-for-sale?search=&auto=&type=&bedroom=&min_price=&max_price=" , headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
soup = BeautifulSoup(webpage, 'html.parser')   
listing_property = soup.find_all("div", class_='listings-property')
for i in listing_property:
    if "sponsored-listing" in i["class"]:
        listing_property.remove(i)


In [18]:

property =  3

agentContact = re.sub("[\n ]", "", listing_property[property].find('div', class_='phone-icon').text)
listingTitile = listing_property[property].find('h3', class_='listings-property-title2').text
listingURL = "https://www.propertypro.ng" + listing_property[property].find("div", "result-img").a['href']
amenities = listing_property[property].find('div', class_='fur-areea').findAll("span", recursive=False)
beds = amenities[0].text[0:1]
bathrooms = amenities[1].text[0:1]
toilets = amenities[2].text[0:1]

price = ''.join(re.findall(r'\d+', listing_property[property].find('h3', class_='listings-price').text))
location = listing_property[property].find('div', class_='result-img').a.img['alt'].split(' ')
street = " ".join(listing_property[property].find('div', class_='single-room-text').findAll("h4")[1].text.split(' ')[0 : -2])
city = location[-2]
state = location[-1]
status = str(re.sub("[\n]", "", listing_property[4].find('div', class_='furnished-btn').text))
if status == "":
    status = "unstated"
dateUpdated = " ".join(listing_property[property].find('div', class_='single-room-text').find("h5").text.split()[-3 :])
imagesProvided =  re.sub("[\n ]", "", listing_property[property].find('div', class_='room-icons').text)
# print([listingTitile, price, bathrooms, beds, toilets, street, city, state, status, agentContact, imagesProvided, listingURL, dateUpdated])
print(street)


Chevron


In [6]:
# CREATE DATAFRAME
header =["listingTitile", "price", "bathrooms", "beds", "toilets", "street", "city", "state", "status", "agentContact", "imagesProvided", "listingURL", "dateUpdated"]
df = pd.DataFrame(data=data, columns=header)

In [7]:
df.head()

,listingTitile,price,bathrooms,beds,toilets,street,city,state,status,agentContact,imagesProvided,listingURL,dateUpdated
0,Newly Renovated & Spacious 4bedoom Duplex (no ...,5000000,5,4,5,Adeniran Ogunsanya,Surulere,Lagos,Newly Built,08167742895,20,https://www.propertypro.ng/property/4-bedroom-...,12 Jul 2022
1,4 Bedroom Semi Detached For Rent,6500000,5,4,5,Osapa London,Lekki,Lagos,Newly Built,09035137760,9,https://www.propertypro.ng/property/4-bedroom-...,12 Jul 2022
2,Newly Built 4 Bedroom Terrace Duplex,6000000,4,4,5,Lekki Phase 1,Lekki,Lagos,Newly Built,07047998445,9,https://www.propertypro.ng/property/4-bedroom-...,05 May 2022
3,4 Bedroom Semi Detached Duplex,6500000,5,4,5,Ologolo,Lekki,Lagos,Newly Built,09035137760,10,https://www.propertypro.ng/property/4-bedroom-...,11 Jul 2022
4,4 Bedroom Terrace Apartment,6000000,5,4,5,Spar Road Just By House On The Rock And Nicon ...,Lekki,Lagos,Newly Built,09035137760,9,https://www.propertypro.ng/property/4-bedroom-...,11 Jul 2022


In [13]:
# CONVERT TO JSON

df.to_json (r'propertyproData' , orient='records',)

In [9]:
json_results
parsed = json.loads(df.to_json(orient="records"))
json.dumps(parsed, indent=4)  

'{"listingTitile":{"0":"Newly Renovated & Spacious 4bedoom Duplex (no Bq)..","1":"4 Bedroom Semi Detached For Rent","2":"Newly Built 4 Bedroom Terrace Duplex","3":"4 Bedroom Semi Detached Duplex","4":"4 Bedroom Terrace Apartment","5":"Mini Flat","6":"4 Bedroom Semi Detached Duplex","7":"2 Bedroom Apartment","8":"3 Bedroom Terrace House","9":"6 Bedroom Duplex","10":"2 Units Of 4 Bedroom Terrace Duplex","11":"Luxury 3 Bedroom Terrace House","12":"5 Bedroom Semi Detached Duplex","13":"19 Units Of Fully Serviced 3 Bedroom Semi Detached Duplex","14":"4 Bedroom Terrace Duplex","15":"3 Bedroom Semi Detached Duplex","16":"Fully Detached 5 Bedroom Duplex","17":"4 Bedroom Fully Detached Duplex","18":"5 Bedroom Detached House","19":"Spacious 4 Bedroom Detached Duplex","20":"4 Bedroom Terrace House","21":"5 Bedroom Detached Duplex","22":"4 Bedroom Semi Detached Duplex","23":"4 Bedroom Semi Detached Duplex","24":"Fully Serviced 3 Bedroom Semi Detached For Rent","25":"4 Bedroom Duplex","26":"4 Bedro